In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     ---------------------------------------- 46.1/46.1 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 0.2/377.1 MB 2.5 MB/s eta 0:02:33
   ---------------------------------------- 0.3/377.1 MB 3.4 MB/s eta 0:01:51
   ---------------------------------------- 0.4/377.1 MB 3.2 MB/s eta 0:01:58
   ---------------------------------------- 0.7/377.1 MB 3.2 MB/s eta 0:01:59
   ---------------------------------------- 0.8/377.1 MB 3.3 MB/s eta 0:01:55
   ---------------------------------------- 1.1/377.1 MB 3.8 MB/s eta 0:01:38
   ---------------------------------------- 1.4/377.1 MB 4.2 MB/s eta 0:01:30
   ---------------------------------------- 1.6/377.1 MB 4.2 MB/s eta 0:01:30
   ---------------------------------------- 1.8/377.1 MB 4.1 MB/s eta 0:01:


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# -*- coding: utf-8 -*-
"""
@author: Robert Kamunde
"""


import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import numpy as np

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

K.clear_session()

n_classes = 10
img_width, img_height = 224, 224
train_data_dir = 'food-101/train'
validation_data_dir = 'food-101/test'
nb_train_samples = 1500
nb_validation_samples = 500
batch_size = 20

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


mbv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape = (224,224,3))
x = mbv2.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(10,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=mbv2.input, outputs=predictions)
opt = SGD(momentum=0.9, learning_rate=0.0001)  # Set learning rate here
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class_sept.keras', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history.log')

history = model.fit(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=10,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained.keras')
    


2.16.1


Found 1500 images belonging to 10 classes.
Found 500 images belonging to 10 classes.
Epoch 1/10


C:\Users\aditi\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1052 - loss: 2.5812
Epoch 1: val_loss improved from inf to 2.32969, saving model to best_model_3class_sept.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.1055 - loss: 2.5802 - val_accuracy: 0.2040 - val_loss: 2.3297
Epoch 2/10

Epoch 2: val_loss improved from 2.32969 to 0.00000, saving model to best_model_3class_sept.keras


C:\Program Files\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2482 - loss: 2.2339
Epoch 3: val_loss did not improve from 0.00000
75/75 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.2485 - loss: 2.2331 - val_accuracy: 0.3140 - val_loss: 2.0415
Epoch 4/10

Epoch 4: val_loss did not improve from 0.00000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3583 - loss: 1.9403
Epoch 5: val_loss did not improve from 0.00000
75/75 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.3586 - loss: 1.9397 - val_accuracy: 0.4020 - val_loss: 1.8116
Epoch 6/10

Epoch 6: val_loss did not improve from 0.00000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.00